In [1]:
from aisdb.ports.api import WorldPortIndexClient # for weather
xmin, ymin, xmax, ymax = -70, 45, -58, 53

# Initialize the client
client = WorldPortIndexClient()

# Query Gulf of St. Lawrence region
# df_ports = client.fetch_ports(ymin, ymax, xmin, xmax, out_path="gulf_ports.csv") # save out to a file
df_ports = client.fetch_ports(46, 51.5, -66.5, -58) 

# Filter for cargo-capable ports
df_cargo = client.filter_by_cargo_depth(df_ports)

Querying WPI with bounds: LATITUDE >= 46 AND LATITUDE <= 51.5 AND LONGITUDE >= -66.5 AND LONGITUDE <= -58
Retrieved 23 ports
1 ports with cargo depth in ('A', 'B', 'C', 'D', 'E', 'F')


In [2]:
df_ports.head()

,OBJECTID,INDEX_NO,REGION_NO,PORT_NAME,COUNTRY,LATITUDE,LONGITUDE,PUB,CHART,HARBORSIZE,...,FUEL_OIL,DIESEL,DECKSUPPLY,ENG_SUPPLY,REPAIRCODE,DRYDOCK,RAILWAY,GlobalID,LAT,LON
0,1574,5520,5501,DOUGLASTOWN,CA,48.766667,-64.383333,145,14186,V,...,,,,,,,,2bef6060-531c-4d1a-97e1-a777830b0375,48.766667,-64.383333
1,1575,5540,5501,CHANDLER,CA,48.366667,-64.666667,145,14186,S,...,Y,Y,Y,Y,C,,,ba9f9827-90df-41df-9910-2f7cf1e6080e,48.366667,-64.666667
2,1576,5600,5555,CARAQUET,CA,47.800000,-64.933333,145,14184,S,...,Y,Y,,,C,,,5be0d191-f6c8-439e-82e0-f0b5fcfbc6b8,47.800000,-64.933333
3,1577,5620,5555,SHIPPEGAN,CA,47.750000,-64.700000,145,14184,V,...,Y,Y,Y,,,,,eb08183f-52e3-4c90-aaca-153c59242f3b,47.750000,-64.700000
4,1578,5680,5555,SHEDIAC,CA,46.216667,-64.550000,145,14002,V,...,Y,Y,,,C,,S,7e8c7e84-b7dc-4fbf-bc23-6d59aed229e8,46.216667,-64.550000
